In [1]:
from time import time

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential

In [2]:
start_time = time()
img_width = 28
img_height = 28
channels = 1
num_classes = 10

# Initialize the model
model = Sequential()

# Add a convolutional layer with 32 kernels, 3x3 kernel size, and ReLU activation function
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, channels)))

# Add a max pooling layer with 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer with 64 kernels, 3x3 kernel size, and ReLU activation function
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add a max pooling layer with 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output to a 1D array
model.add(Flatten())

# Add a fully connected layer with 128 neurons and ReLU activation function
model.add(Dense(128, activation='relu'))

# Add a dropout layer with a 0.5 dropout rate
# model.add(Dropout(0.5))

# Add the output layer with a softmax activation function for multi-class classification
model.add(Dense(num_classes, activation='softmax'))

# Compile the model with categorical cross-entropy loss function and Adam optimizer with a learning rate of 0.001
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

end_time = time()
print("Time taken to build the model: {} seconds".format(end_time - start_time))
# Print a summary of the model architecture
model.summary()



2023-03-20 13:12:41.109375: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-20 13:12:41.109397: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro
Time taken to build the model: 0.6899950504302979 seconds
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                              